# 掩码语言模型训练实例

## Step1 导入相关包

In [1]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")


def process_func(examples):
    return tokenizer(
        examples["completion"], max_length=384, truncation=True
    )  # mask操作和labels在DataCollatorForLanguageModeling

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(
    tokenized_ds,
    batch_size=2,
    collate_fn=DataCollatorForLanguageModeling(
        tokenizer, mlm=True, mlm_probability=0.15
    ),
)

In [8]:
next(enumerate(dl))
# input_ids里103是被mask的
# labels里非-100的数据是被mask的

(0,
 {'input_ids': tensor([[  101,  6205,  2128,   769,  6858,  1920,  2110,  1300, 15688,  7667,
           8020, 13135,   112,  9064,   103,  8731,  8626,  8181,   103, 10553,
           8021,  3221,   671,  2429,   855,   754,  6205,  2128,   769,  6858,
           1920,  2110,  4638,  1300,  4289,  7667,  8024,  7667,  7270,  3221,
           7247,  3209,  1587,   511,  1325,  1380,  8258,  2399,   130,   103,
           8113,  3189,  2458,  1993,  5040,  2456,  8024,  8138,  2399,   125,
           3299,   129,  3189,  3633,  2466,  2456,  2768,  2458,   103,  8024,
            855,   754,   103,   103,   103,  6858,  1920,  2110,  1069,   103,
           3413,  1277,  7362,  6205,  4689,  6205,  2128,  2356,  1496,  2123,
           6205,  6662,  8143,  1384,   511,  2456,  5029,  7481,  4916,   127,
            117,  8280,  2398,  5101,  8024,  2245,  1324,  7481,  4916,   125,
            103,  8195,  2398,  5101,  8024,  7667,  5966,  3152,  4289,   125,
            117,  8567

In [9]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

## Step4 创建模型

In [10]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Step5 配置训练参数

In [11]:
args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1,
)

## Step6 创建训练器

In [12]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer, mlm=True, mlm_probability=0.15
    ),
)

## Step7 模型训练

In [13]:
trainer.train()

  3%|▎         | 10/313 [00:11<05:37,  1.11s/it]

{'loss': 1.4016, 'grad_norm': 3.5025999546051025, 'learning_rate': 4.840255591054313e-05, 'epoch': 0.03}


  6%|▋         | 20/313 [00:22<05:19,  1.09s/it]

{'loss': 1.4074, 'grad_norm': 3.2377915382385254, 'learning_rate': 4.680511182108626e-05, 'epoch': 0.06}


 10%|▉         | 30/313 [00:33<05:09,  1.09s/it]

...

 99%|█████████▉| 310/313 [05:40<00:03,  1.12s/it]

{'loss': 1.2611, 'grad_norm': 3.3543589115142822, 'learning_rate': 4.792332268370607e-07, 'epoch': 0.99}


100%|██████████| 313/313 [05:43<00:00,  1.10s/it]

{'train_runtime': 343.8703, 'train_samples_per_second': 29.081, 'train_steps_per_second': 0.91, 'train_loss': 1.327999483663053, 'epoch': 1.0}


TrainOutput(global_step=313, training_loss=1.327999483663053, metrics={'train_runtime': 343.8703, 'train_samples_per_second': 29.081, 'train_steps_per_second': 0.91, 'total_flos': 1973819658240000.0, 'train_loss': 1.327999483663053, 'epoch': 1.0})

## Step8 模型推理

In [14]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [15]:
pipe(
    "西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆"
)

[[{'score': 0.999140739440918,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0004397405427880585,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 4.7670037019997835e-05,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 1.8455242752679624e-05,
   'token': 7770,
   'token_str': '高',
   'sequence': "[CLS] 西 安 交 通 高 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 1.606051591807045e-05,
   'token': 3413,
   'token_str': '校',
   'sequence': "[CLS] 西 安 交 通 校 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.998818814754

In [16]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！")

[[{'score': 0.12022993713617325,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.08464808762073517,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.07992567121982574,
   'token': 4178,
   'token_str': '热',
   'sequence': '[CLS] 下 面 是 一 则 热 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.052737388759851456,
   'token': 3173,
   'token_str': '新',
   'sequence': '[CLS] 下 面 是 一 则 新 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.03921257704496384,
   'token': 4685,
   'token_str': '相',
   'sequence': '[CLS] 下 面 是 一 则 相 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.09280014783143997,
   'token': 6206,
   'token_str': '要',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 要 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  